# 데이터 가져오기

In [1]:
import pandas as pd
import glob
import csv

path = './data/albam/'
files = glob.glob(path + "/*.csv")
li = []

for file in files:
    df = pd.read_csv(file, sep=',', escapechar='\n')
    li.append(df)
    
albam = pd.concat(li, axis=0, ignore_index=True)
print(albam.shape)
albam.head()

(25, 9)


,이름,시급,연장근무,야간근무,휴일근무,근무인정시간,총급여,기본급여,주휴수당
0,백정민(690125),8590,NaN,NaN,NaN,4시간46분,40946,40946.0,0.0
1,이웅(692649),8590,NaN,NaN,NaN,5시간9분,44239,44239.0,0.0
2,김정건(688307),8590,NaN,NaN,NaN,2시간1분,17323,17323.0,0.0
3,백정민(690125),8590,NaN,NaN,NaN,-,0,0.0,0.0
4,신수정(688324),8590,NaN,NaN,NaN,2시간24분,20616,20616.0,0.0


# 원본 데이터 숫자화

In [2]:
computed = pd.DataFrame()
computed['이름'] = albam['이름'].apply(lambda row: row[:row.find('(')])
computed['시급'] = albam['시급'].astype(int)
computed['근무hrs'] = albam['근무인정시간'].apply(lambda row: row[:row.find('시간')] if row != '-' else 0).astype(int)
computed['근무mins'] = albam['근무인정시간'].apply(lambda row: row[row.find('간')+1:row.find('분')] if row != '-' else 0).astype(int)
computed['기본급'] = albam['기본급여'].astype(float)
computed.head()

,이름,시급,근무hrs,근무mins,기본급
0,백정민,8590,4,46,40946.0
1,이웅,8590,5,9,44239.0
2,김정건,8590,2,1,17323.0
3,백정민,8590,0,0,0.0
4,신수정,8590,2,24,20616.0


# 초기 데이터 통계

In [21]:
df = pd.DataFrame()
df['이름'] = computed['이름'].unique()

for idx, row in computed.iterrows():
    name = row['이름']
    hrs = computed[computed['이름']==name]['근무hrs'].sum()
    mins = computed[computed['이름']==name]['근무mins'].sum()
    if mins>60:
        hrs += 1
        mins -= 60
    total_time_in_mins = (hrs*60) + mins
    gross_salary = row['시급'] * total_time_in_mins
    insurance = gross_salary * 0.009628
    
    df.loc[df['이름']==name,'일한시간'] = f"{hrs}시간 {mins}분"
    df.loc[df['이름']==name,'기본급'] =  gross_salary
    df.loc[df['이름']==name,'4대보험'] = round(insurance, 2)
    df.loc[df['이름']==name,'급여'] = round(gross_salary - insurance)
df

,이름,일한시간,기본급,4대보험,급여
0,백정민,31시간 171분,17446290.0,167972.88,17278317.0
1,이웅,41시간 66분,21698340.0,208911.62,21489428.0
2,김정건,10시간 59분,5660810.0,54502.28,5606308.0
3,신수정,13시간 1분,6708790.0,64592.23,6644198.0
